<a href="https://colab.research.google.com/blob/github/PaulSZH95/hs_peer_support_chem/blob/main/HS_science_A_student_simulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instruct-tuning Open LLaMA using sciq and HydraLM

### This tutorial is inspired by:
- DeepLearningAI [webinar](https://www.youtube.com/watch?v=eTieetk2dSw&t)
- [Speakers' repository](https://github.com/FourthBrain/Building-with-Instruction-Tuned-LLMs-A-Step-by-Step-Guide)

#### Motivation:
Harness the power of LLM to simulate a high school peer tutor (universal) for chemistry students.

#### [SCIq](https://arxiv.org/pdf/1707.06209.pdf):
Dataset created in 2017 leveraging a large corpus of 28 science study textbooks from various online learning resources, including CK-12 and OpenStax, which are shared under a Creative Commons License.
The dataset is said to provide better question and answer qualities.

#### [Chemistry_dataset_standardized](https://docs.google.com/document/d/1YKDRCu7M9mflWrxKc1HeFs2HBWk4HXVrrOsLrIn6EXM/edit):
Dataset created by hydraml for the purpose of finetuning LLAMA 2 to achieve Chatgpt4 capabilities.

#### [LORA](https://arxiv.org/pdf/2106.09685.pdf):
- Any matrix can be decompiosed to 2 matrics: matrics of A x B can be a product of Matrix A.r X r.B
In this case, r is the rank and you can speed findtuning up by finetuning from small r up to larger r. (this is a optuna job)

#### [QLORA](https://huggingface.co/docs/optimum/concept_guides/quantization):
- Now you do it with a quantized mechanism using a datatype of 4-bit NormalFloat (NF)

#### [PEFT](https://huggingface.co/blog/peft):
A library for you to inject layers before/after the transformer layers.

#### Summary:
We are only training the injected layers (adapters) and we are using LORA technique and 4bit quantization to speed things up.

In [ ]:
!pip install -q -U git+https://github.com/lvwerra/trl.git
!pip install -q -U bitsandbytes
!pip install -U -q git+https://github.com/huggingface/transformers@de9255de27abfcae4a1f816b904915f0b1e23cd9
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U sentencepiece

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requiremen

Let's look at our dataset to get an idea of what we're working with!

In [ ]:
import datasets
from datasets import load_dataset, DatasetDict

sciq = load_dataset("sciq")
chem_stand = load_dataset("HydraLM/chemistry_dataset_standardized")

Generating train split:   0%|          | 0/11679 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# metadata of datasets
print(sciq)
print("\n\n")
print(chem_stand)

DatasetDict({
    train: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 11679
    })
    validation: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 1000
    })
})



DatasetDict({
    train: Dataset({
        features: ['message', 'message_type', 'message_id', 'conversation_id'],
        num_rows: 40000
    })
})


## HydraLM/chemistry_dataset_standardized
See first 4 output of dataset to realise that the Question is the first entrant of the message key, the answer be the subsequent entrant followed by new question and answers.

We will restructure the dataset for our use case



In [ ]:
# first 2 entrant is displayed.
print(chem_stand["train"][0])
print('\n')
print(chem_stand["train"][1])

{'message': 'Calculate the standard enthalpy of formation of C2H6(g) given the following information: \n\nC(s) + O2(g) → CO2(g); ΔH° = -393.5 kJ/mol \n\nH2(g) + 1/2 O2(g) → H2O(l); ΔH° = -285.8 kJ/mol \n\n2C(s) + 3H2(g) → C2H6(g); ΔH° = ?', 'message_type': 'input', 'message_id': 0, 'conversation_id': 0}


{'message': "To calculate the standard enthalpy of formation of C2H6(g), we need to use Hess's Law, which states that the total enthalpy change of a reaction is the sum of the enthalpy changes of its individual steps.\n\nFirst, we need to manipulate the given reactions to match the target reaction:\n\nTarget reaction: 2C(s) + 3H2(g) → C2H6(g)\n\n1. 2(C(s) + O2(g) → CO2(g)); ΔH° = 2(-393.5 kJ/mol) = -787 kJ/mol\n2. 3(H2(g) + 1/2 O2(g) → H2O(l)); ΔH° = 3(-285.8 kJ/mol) = -857.4 kJ/mol\n3. 2CO2(g) + 3H2O(l) → C2H6(g) + 4O2(g); ΔH° = ?\n\nNow, we can add reactions 1 and 2 and reverse reaction 3 to obtain the target reaction:\n\n2C(s) + 2O2(g) + 3H2(g) + 3/2 O2(g) → 2CO2(g) + 3H2O(l) + C2H

In [ ]:
# now we restructure dataset
import datasets
questions = []
response = []
for i,j in enumerate(chem_stand["train"]["message"]):
  if i % 2 == 0:
    questions.append(j)
  else:
    response.append(j)
chem_message_dict = datasets.Dataset.from_dict({"questions":questions,"answers":response})
chem_dataset = DatasetDict({
    "train": chem_message_dict
})

In [ ]:
# split chem_stand to train 80, val 10, test 10
train_test = chem_dataset["train"].train_test_split(test_size=0.2)
test_valid = train_test['test'].train_test_split(test_size=0.5)
chem_dataset_split = DatasetDict({
    'train': train_test['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']})
chem_dataset_split

DatasetDict({
    train: Dataset({
        features: ['questions', 'answers'],
        num_rows: 16000
    })
    test: Dataset({
        features: ['questions', 'answers'],
        num_rows: 2000
    })
    validation: Dataset({
        features: ['questions', 'answers'],
        num_rows: 2000
    })
})

In [ ]:
# train set is pretty large for both sciq and chem_stand, not suitable for free colab training. Will reduce it to N entries

def subsampling(dataset_obj_keyed, n_start,n_end):
  return datasets.Dataset.from_dict(dataset_obj_keyed[n_start:n_end])


In [ ]:
train_2000 = subsampling(chem_dataset_split["train"],100,2100)
train_2000_chem = DatasetDict({
    'train': train_2000,
    'test': chem_dataset_split['test'],
    'validation': chem_dataset_split['validation']})
train_2000_chem

DatasetDict({
    train: Dataset({
        features: ['questions', 'answers'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['questions', 'answers'],
        num_rows: 2000
    })
    validation: Dataset({
        features: ['questions', 'answers'],
        num_rows: 2000
    })
})

In [ ]:
train_2k = subsampling(sciq["train"],100,2100)
train_2000_sciq = DatasetDict({
    'train': train_2k,
    'test': sciq['test'],
    'validation': sciq['validation']})
train_2000_sciq

DatasetDict({
    train: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 1000
    })
})

In [ ]:
# intruction formating for chem_standard
import random
def chem_format(example):
  question = example.get("questions","")
  answer = example.get("answers","")
  starter = "Below is a question your task is to provide an answer \n"
  question_part = f"### Question: {question} \n\n"
  answer_part = f"### Answer: {question} \n\n"
  full_prompt = f"{starter}{question_part}{answer_part}"
  return {"text": full_prompt}

def sciq_format(example):
  question = example.get("question","")
  op1 = example.get('distractor3',"")
  op2 = example.get('distractor2',"")
  op3 = example.get('distractor1',"")
  ans = example.get('correct_answer',"")
  options = [op1,op2,op3,ans]
  random.shuffle(options)
  options = ",".join(options)
  support = example.get("support","")
  starter = f"Below is a question with options options provided. Only 1 out of 4 options will be correct. your task is to chose a correct answer from given options and provide support \n\n"
  question_part = f"### Question: {question} \n\n"
  options_part = f"### Options: {options} \n\n"
  answer_part = f"### Answer: {ans} \n\n"
  support_part = f"### Support: {support}"
  full_prompt = f"{starter}{question_part}{options_part}{answer_part}{support_part}"
  return {"text": full_prompt}

In [ ]:
chem_formated = train_2000_chem.map(chem_format)
sciq_formated = train_2000_sciq.map(sciq_format)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
print(chem_formated)
print(chem_formated["train"][0]["text"])
print(sciq_formated)
print(sciq_formated["train"][0]["text"])

DatasetDict({
    train: Dataset({
        features: ['questions', 'answers', 'text'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['questions', 'answers', 'text'],
        num_rows: 2000
    })
    validation: Dataset({
        features: ['questions', 'answers', 'text'],
        num_rows: 2000
    })
})
Below is a question your task is to provide an answer 
### Question: What is the concentration of protein in a given sample of milk using spectrophotometry technique? 

### Answer: What is the concentration of protein in a given sample of milk using spectrophotometry technique? 


DatasetDict({
    train: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support', 'text'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support', 'text'],
        num_rows: 1000
    })
    validation: Dataset({
        features: [

In [ ]:
# combine datasets

def list_to_text_datadict(list_you_made):
  random.shuffle(list_you_made)
  return datasets.Dataset.from_dict({"text": list_you_made})


comb_train = sciq_formated['train']["text"] + chem_formated['train']["text"]
comb_test = sciq_formated['test']["text"] + chem_formated['test']["text"]
comb_val  = sciq_formated['validation']["text"] + chem_formated['validation']["text"]

# shuffle

comb_dataset = DatasetDict({
    'train': list_to_text_datadict(comb_train),
    'test': list_to_text_datadict(comb_test),
    'validation': list_to_text_datadict(comb_val)})
comb_dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['text'],
        num_rows: 3000
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3000
    })
})

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
import transformers
from peft import LoraConfig
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer

model_id = "h2oai/h2ogpt-gm-oasst1-en-2048-open-llama-7b-preview-700bt"

qlora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    use_auth_token=True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from transformers import LlamaTokenizer

tokenizer = LlamaTokenizer.from_pretrained(model_id)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [ ]:
print(base_model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )


## SFFT Model:

In [ ]:
from trl import SFTTrainer

supervised_finetuning_trainer = SFTTrainer(
    base_model,
    train_dataset=comb_dataset["train"],
    eval_dataset=comb_dataset["test"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        learning_rate=2e-4,
        max_steps=1000,
        output_dir="./tune-LLama",
        optim="paged_adamw_8bit",
        fp16=True,
    ),
    tokenizer=tokenizer,
    peft_config=qlora_config,
    dataset_text_field="text",
    max_seq_length=512
)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

### Observe adapters injected into the LLAMA model.
#### Everything about the original model is frozen

In [ ]:
base_model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(
            in_features=4096, out_features=4096, bias=False
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(
            in_features=4096, out_features=4096, bias=False
            (lora_dropout): ModuleDict(
              (default):

In [ ]:
supervised_finetuning_trainer.train()

Step,Training Loss
500,0.833300
1000,0.773200


TrainOutput(global_step=1000, training_loss=0.8032535400390625, metrics={'train_runtime': 3987.1041, 'train_samples_per_second': 1.003, 'train_steps_per_second': 0.251, 'total_flos': 1.230655490064384e+16, 'train_loss': 0.8032535400390625, 'epoch': 1.0})

In [ ]:
repo = 'supramantest/hs_peer_support_chem'
NEW_MODEL_NAME = 'chem_tutor'
supervised_finetuning_trainer.model.save_pretrained(NEW_MODEL_NAME)
lora_config = LoraConfig.from_pretrained(NEW_MODEL_NAME)

In [ ]:
peft_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
              

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# you must follow exactly , less the repo name, as the rapid changes to the libraries might break stuff.
# raise issue on github if anything breaks
from peft import PeftModel, get_peft_model
repo = 'supramantest/hs_peer_support_chem'
peft_model = get_peft_model(base_model, lora_config)
peft_model.push_to_hub(repo)
tokenizer.push_to_hub(repo)
comb_dataset.push_to_hub(repo)

CommitInfo(commit_url='https://huggingface.co/supramantest/hs_peer_support_chem/commit/ab643fc28841ceee7e74f639fc15ef93a464f06f', commit_message='Upload model', commit_description='', oid='ab643fc28841ceee7e74f639fc15ef93a464f06f', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# inference part can be run separately from the training part
from peft import get_peft_model
import torch
import transformers
from peft import LoraConfig
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, LlamaTokenizer


lora_config = LoraConfig.from_pretrained("supramantest/hs_peer_support_chem")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = LlamaTokenizer.from_pretrained("supramantest/hs_peer_support_chem")
model = AutoModelForCausalLM.from_pretrained(
    lora_config.base_model_name_or_path,
    quantization_config=bnb_config,
    device_map={"":0})

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
model = get_peft_model(model, lora_config)

In [ ]:
from IPython.display import display, Markdown

def make_inference(prompt, context = None):
  inputs = tokenizer(prompt, return_tensors="pt", return_token_type_ids=False).to("cuda:0")
  outputs = model.generate(**inputs, max_new_tokens=100)
  display(Markdown((tokenizer.decode(outputs[0], skip_special_tokens=True))))
  outputs = model.generate(**inputs, max_new_tokens=50)
  print("---- NON-INSTRUCT-TUNED-MODEL ----")
  display(Markdown((tokenizer.decode(outputs[0], skip_special_tokens=True))))

In [ ]:
### Answer: neurotransmitters ### Support: When a nerve impulse reaches the end of an axon, the axon releases chemicals called neurotransmitters .

In [ ]:
prompt1 = "Below is a question with options options provided. Only 1 out of 4 options will be correct. your task is to chose a correct answer from given options and provide support ### Question: When a nerve impulse reaches the end of an axon, what are the chemicals released by the axon called? ### Options: neurons,electrolytes,neurotransmitters,receptors"

make_inference(prompt1)

Below is a question with options options provided. Only 1 out of 4 options will be correct. your task is to chose a correct answer from given options and provide support ### Question: When a nerve impulse reaches the end of an axon, what are the chemicals released by the axon called? ### Options: neurons,electrolytes,neurotransmitters,receptors

### Answer: Neurotransmitters

The correct answer is neurotransmitters.

Neurotransmitters are chemicals released by neurons to communicate with other neurons. They are responsible for transmitting signals across the synapse, which is the gap between two neurons.

### Explanation:

The correct answer is neurotransmitters. Neurotransmitters are chemicals released by neurons to communicate with other neurons. They are responsible for transmitting signals

---- NON-INSTRUCT-TUNED-MODEL ----


Below is a question with options options provided. Only 1 out of 4 options will be correct. your task is to chose a correct answer from given options and provide support ### Question: When a nerve impulse reaches the end of an axon, what are the chemicals released by the axon called? ### Options: neurons,electrolytes,neurotransmitters,receptors

### Answer: Neurotransmitters

The correct answer is neurotransmitters.

Neurotransmitters are chemicals released by neurons to communicate with other neurons. They are responsible for transmitting signals across the synapse